# Finding alpha signals in insider trading

In this post, we’ll take a look at insider trading (the legal kind, not the illegal kind) and what information, if any, investors can draw from the required disclosures that insiders must make.

In [2]:
import pandas as pd
from fidap import fidap_client
import config

## What is insider trading

Insider trading generally has a negative connotation of being illegal. However, that’s most often not the case - the vast majority of insider trading is perfectly legal. An “insider” is any individual that may have material nonpublic information about a company. Generally, the definition includes the following people - 

1. Company directors
2. Company officers
3. Any shareholder with >10% holdings
4. Often, corporate insiders will need to trade in the shares of the company. Whenever an insider transacts in the shares of the company, they must file a declaration with the SEC called a Form 4. A filing looks something like this - 

More detailed information about what these fields mean can be found here.


## Is there information in insider trading?

For investors, the key question is whether or not insider trades can be used as a signal. There are two key questions we need to ask ourselves - 

1. What is the reason behind the insider trade?
2. If the reason behind the insider trade is price-related (eg an opinion on whether the price will go up or down), are insiders accurate?

There could be a variety of reasons that insiders trade in a security beyond whether they think the price is going to go up or down. For company directors and officers, a large part of their compensation may be based in shares, and they may decide to liquidate those shares for diversification or for liquidity needs for other reasons, such as taxes or other purchases. Though less common, even insider buys may have motivations driven by requirements, such as the need to maintain a certain level of ownership.

However, if the motivation of the insider trade is indeed an opinion on the future trajectory of the price of the stock, then we may conclude that insiders have additional information about the future prospects of the company. 

Therefore, can we assume that insiders buying shares is a positive signal and insiders selling shares is a negative signal?



## Data exploration using Fidap

Let’s see what the data tells us. Fidap has a dataset around insider trades that covers all SEC filings. Here’s a link to the table details page. There are 24 columns and 11.9m rows. The columns include information on the date of the transaction and the date of the filing, the transaction amount, the owner, the underlying security, as well as several other details.

The columns are below - 



In [4]:
fidap = fidap_client(api_key=config.FIDAP_API_KEY)

In [5]:
fidap.sql("""
    select * from meta_fields where "table" = 'insider_trades'
""")

,table,name,display_name,description,pd_type,pct_filled,uniqueness,mean,median
0,insider_trades,ticker,Ticker Symbol,The ticker is a unique identifer for an issuer...,object,1.000000,1.516150e-03,NaN,NaN
1,insider_trades,filingdate,Filing Date,The date the form was filed with the SEC.,datetime64[ns],1.000000,3.774522e-04,NaN,NaN
2,insider_trades,formtype,Form Type,"""The type of SEC form . Available options are ...",object,1.000000,6.917000e-07,NaN,NaN
3,insider_trades,issuername,Issuer Name,The name of the security issuer.,object,0.990252,1.494591e-03,NaN,NaN
4,insider_trades,ownername,Owner Name (Insider / Investor),The name of the owner.,object,1.000000,1.886212e-02,NaN,NaN
5,insider_trades,officertitle,Officer Title,Is the owner is an officer of the company the ...,object,0.606570,7.799448e-03,NaN,NaN
6,insider_trades,isdirector,Is Director?,Is the owner a Board Director? [Y]es or [N]o.,object,1.000000,2.306000e-07,NaN,NaN
7,insider_trades,isofficer,Is Officer?,Is the owner an officer of the company? [Y]es ...,object,1.000000,2.306000e-07,NaN,NaN
8,insider_trades,istenpercentowner,Is Ten Percent Owner?,Does the owner hold ten percent or more of the...,object,1.000000,2.306000e-07,NaN,NaN
9,insider_trades,transactiondate,Transaction Date,If there has been a transaction; the date of t...,datetime64[ns],0.688095,7.890295e-04,NaN,NaN


Over fifteen years that we have data in Fidap, we see a total 12 million transactions, covering 15,000 tickers, and a whopping $9 trillion in transaction volume. See the query below

In [4]:
fidap.sql("""
    select count(*) as count, sum(transactionvalue) as tvalue, 
    count(distinct(ticker)) as tickers, min(filingdate) from insider_trades
""")

,COUNT,TVALUE,TICKERS,MIN(FILINGDATE)
0,7223907,6669452022398,13120,2008-01-02


Let's first explore the data and find some basic numbers here.

We can see these same numbers in the Fidap dashboard as well.

Let's see some of the individual rows here.

In [5]:
fidap.sql("""
    select * from insider_trades where ticker='STT'  order by filingdate desc limit 5
""")

,TICKER,FILINGDATE,FORMTYPE,ISSUERNAME,OWNERNAME,OFFICERTITLE,ISDIRECTOR,ISOFFICER,ISTENPERCENTOWNER,TRANSACTIONDATE,SECURITYADCODE,TRANSACTIONCODE,SHARESOWNEDBEFORETRANSACTION,TRANSACTIONSHARES,SHARESOWNEDFOLLOWINGTRANSACTION,TRANSACTIONPRICEPERSHARE,TRANSACTIONVALUE,SECURITYTITLE,DIRECTORINDIRECT,NATUREOFOWNERSHIP,DATEEXERCISABLE,PRICEEXERCISABLE,EXPIRATIONDATE,ROWNUM
0,STT,2020-11-20,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,2020-11-19,ND,S,18704,-400.0,18304,68.59,27436.0,Common Stock,D,None,NaN,NaN,NaN,1
1,STT,2020-11-20,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,None,N,None,550,NaN,550,NaN,NaN,Common Stock,I,By domestic partner,NaN,NaN,NaN,2
2,STT,2020-11-17,4,STATE STREET CORP,PHELAN DAVID C,EVP; Gen Counsel and Secretary,N,Y,N,2020-11-15,ND,F,98071,-1822.0,96249,68.52,124843.0,Common Stock,D,None,NaN,NaN,NaN,1
3,STT,2020-11-17,4,STATE STREET CORP,TARAPOREVALA CYRUS,EVP; President and CEO of SSGA,N,Y,N,2020-11-15,ND,F,66730,-566.0,66164,68.52,38782.0,Common Stock,D,None,NaN,NaN,NaN,1
4,STT,2020-11-17,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,2020-11-15,ND,F,19801,-781.0,19020,68.52,53514.0,Common Stock,D,None,NaN,NaN,NaN,1


We can actually get info on each of the columns from the metadata table. Let's take a look.

In [6]:
fidap.sql("""
    select * from insider_trades where ticker='AAPL' and securitytitle = 'Common Stock' limit 5
""")

,TICKER,FILINGDATE,FORMTYPE,ISSUERNAME,OWNERNAME,OFFICERTITLE,ISDIRECTOR,ISOFFICER,ISTENPERCENTOWNER,TRANSACTIONDATE,SECURITYADCODE,TRANSACTIONCODE,SHARESOWNEDBEFORETRANSACTION,TRANSACTIONSHARES,SHARESOWNEDFOLLOWINGTRANSACTION,TRANSACTIONPRICEPERSHARE,TRANSACTIONVALUE,SECURITYTITLE,DIRECTORINDIRECT,NATUREOFOWNERSHIP,DATEEXERCISABLE,PRICEEXERCISABLE,EXPIRATIONDATE,ROWNUM
0,AAPL,2018-02-20,4,APPLE INC,KONDO CHRIS,Principal Accounting Officer,N,Y,N,2018-02-15,None,M,8619,675,9294,NaN,NaN,Common Stock,D,NaN,NaN,NaN,NaN,1
1,AAPL,2018-02-20,4,APPLE INC,KONDO CHRIS,Principal Accounting Officer,N,Y,N,2018-02-15,ND,F,9294,-229,9065,172.99,39615.0,Common Stock,D,NaN,NaN,NaN,NaN,2
2,AAPL,2016-02-17,4,APPLE INC,KONDO CHRIS,Principal Accounting Officer,N,Y,N,2016-02-15,None,M,12651,675,13326,NaN,NaN,Common Stock,D,NaN,NaN,NaN,NaN,1
3,AAPL,2016-02-17,4,APPLE INC,KONDO CHRIS,Principal Accounting Officer,N,Y,N,2016-02-15,ND,F,13326,-276,13050,93.99,25941.0,Common Stock,D,NaN,NaN,NaN,NaN,2
4,AAPL,2015-02-20,4,APPLE INC,JUNG ANDREA,None,Y,N,N,2015-02-18,None,M,14595,40000,54595,25.72,1028800.0,Common Stock,D,NaN,NaN,NaN,NaN,1


In [7]:
# need to link with tickers table
fidap.sql("""
    select ticker, count(*) as count, sum(transactionvalue) as tvalue from insider_trades
    where transactionvalue > 0 group by ticker order by tvalue desc limit 10
""")

,ticker,count,tvalue
0,NGBL,995,1425086696067
1,GGP,497,73606075821
2,PWRM,75,61806147612
3,TELOZ,63,50915143904
4,HLT,363,50495362012
5,AMGTI,348,49325889352
6,INVH,347,41747721180
7,OMAG,51,41030154474
8,CHTRQ,864,40386743764
9,FB,6877,40168004044


In [8]:
fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionvalue from insider_trades, tickers
    where insider_trades.ticker = tickers.ticker and insider_trades.transactionvalue > 100000 and insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
    limit 100
""")

,ticker,filingdate,transactionvalue
0,QCOM,2008-03-04,148207
1,QCOM,2008-03-04,535507
2,QCOM,2008-03-04,1050000
3,QCOM,2008-03-04,222300
4,QCOM,2008-03-04,420000
...,...,...,...
95,FSLR,2008-03-04,689822
96,FSLR,2008-03-04,226501
97,FSLR,2008-03-04,954079
98,FSLR,2008-03-04,123300


In [9]:
df = fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionshares,
    insider_trades.transactionvalue, daily.close, daily.fc_1m
    from insider_trades, tickers, daily
    where insider_trades.ticker = 'AAPL' and
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
""")

In [10]:
df

,ticker,filingdate,transactionshares,transactionvalue,close,fc_1m
0,AAPL,2016-02-03,-2008,191704,24.087,0.057334
1,AAPL,2016-03-08,-15088,1554215,25.258,0.075501
2,AAPL,2016-03-17,-18241,1895605,26.450,0.010473
3,AAPL,2016-03-23,-256356,27150664,26.532,-0.009875
4,AAPL,2016-03-23,-256356,27150664,26.532,-0.009875
...,...,...,...,...,...,...
357,AAPL,2020-10-19,-12452,1495983,115.980,0.017675
358,AAPL,2020-11-05,-2612,285283,119.030,0.044947
359,AAPL,2020-11-05,-8641,953966,119.030,0.044947
360,AAPL,2020-11-05,-5747,637860,119.030,0.044947


In [12]:
df['fc_1m'].mean()

0.030035378544751374

In [15]:
df[df['transactionshares'] > 0]['fc_1m'].mean()

0.04398710701875

In [16]:
df[df['transactionshares'] < 0]['fc_1m'].mean()

0.029390211910115607

In [17]:
len(df[df['transactionshares'] > 0]['fc_1m'])

16

In [18]:
df2 = fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionshares,
    insider_trades.transactionvalue, daily.close, daily.fc_1m
    from insider_trades, tickers, daily
    where 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
""")

In [19]:
df2[df2['transactionshares'] > 0]['fc_1m'].mean()

0.016726617027223743

In [20]:
df2[df2['transactionshares'] < 0]['fc_1m'].mean()

0.019570244880066368

In [21]:
fidap.sql("""
    select avg(daily.fc_3m), avg(daily.fc_1m), count(daily.fc_3m)
    from insider_trades, tickers, daily
    where 
    insider_trades.transactionshares > 0 and 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock'
""")

,AVG(DAILY.FC_3M),AVG(DAILY.FC_1M),COUNT(DAILY.FC_3M)
0,0.060503,0.017141,70341


In [12]:
fidap.sql("""
    select avg(daily.fc_3m), avg(daily.fc_1m), count(daily.fc_3m)
    from insider_trades, tickers, daily
    where 
    insider_trades.transactionshares < 0 and 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
""")

,AVG(DAILY.FC_3M),AVG(DAILY.FC_1M),COUNT(DAILY.FC_3M)
0,0.056397,0.00895,174460


In [23]:
fidap.sql("""select * from fundamentals where ticker='URBN' and dimension = 'MRQ' order by calendardate desc""")

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,rev_yoy,rev_qoq
0,URBN,MRQ,2020-09-30,2020-10-31,2020-10-31,2020-12-11,-30894000,3530675000,NaN,1371561000,...,98583032,9.916,3530675000,117705000,20914000,0,36.107,424663000,-0.018089,0.207081
1,URBN,MRQ,2020-06-30,2020-07-31,2020-07-31,2020-12-11,-29203000,3425278000,NaN,1270966000,...,98104918,8.215,3425278000,121292000,34486000,0,35.031,499750000,-0.165290,0.364977
2,URBN,MRQ,2020-03-31,2020-04-30,2020-04-30,2020-12-11,-40925000,3356204000,NaN,1176928000,...,97910314,6.010,3356204000,169054000,-60131000,0,34.278,513351000,-0.319211,-0.496842
3,URBN,MRQ,2019-12-31,2020-01-31,2020-01-31,2020-12-11,-28004000,3315633000,NaN,1053396000,...,98913633,11.940,3315633000,104578000,20077000,0,33.848,414626000,0.035989,0.184420
4,URBN,MRQ,2019-09-30,2019-10-31,2019-10-31,2020-12-11,-29691000,3320593000,NaN,1113013000,...,98628169,10.079,3320593000,114641000,20193000,0,33.893,401961000,0.014315,0.026124
5,URBN,MRQ,2019-06-30,2019-07-31,2019-07-31,2020-12-11,-37287000,3138045000,NaN,1000397000,...,99602465,9.711,3138045000,105814000,21239000,0,31.667,374264000,-0.030354,0.113275
6,URBN,MRQ,2019-03-31,2019-04-30,2019-04-30,2020-12-11,-30717000,3251820000,NaN,1139297000,...,105340148,8.277,3251820000,101267000,10115000,0,31.137,491118000,0.010196,-0.234320
7,URBN,MRQ,2018-12-31,2019-01-31,2019-01-31,2020-12-11,-27103000,2160515000,NaN,1202756000,...,108376713,10.540,2160515000,104438000,28973000,0,20.172,816112000,0.036570,0.159640
8,URBN,MRQ,2018-09-30,2018-10-31,2018-10-31,2020-12-11,-32093000,2197042000,NaN,1248799000,...,110262879,8.950,2197042000,103327000,20072000,0,20.197,793826000,0.090459,-0.019065
9,URBN,MRQ,2018-06-30,2018-07-31,2018-07-31,2020-12-11,-26601000,2158535000,NaN,1201768000,...,110433840,9.119,2158535000,104169000,25789000,0,19.834,771830000,0.136921,0.159832


In [13]:
fidap.sql("""select * from meta_fields where table_name = 'FUNDAMENTALS'""")

,table_name,database_name,schema_name,column_name,count,distinct_count,distinct_count_with_nan,distinct_count_without_nan,freq,max,...,sum,type,variance,table,indicator,isfilter,isprimarykey,title,description,unittype
0,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,ACCOCI,425239,39468,39469,39468,NaN,8.380761e+12,...,9.678876e+13,Variable.TYPE_NUM,2.092612e+21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,ASSETS,425222,95928,95929,95928,NaN,5.524196e+14,...,7.151366e+16,Variable.TYPE_NUM,3.719295e+25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,ASSETSAVG,227911,102644,102645,102644,NaN,5.060100e+14,...,6.331898e+16,Variable.TYPE_NUM,6.390429e+25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,ASSETSC,337341,73325,73326,73325,NaN,4.770100e+13,...,6.812302e+15,Variable.TYPE_NUM,3.579971e+23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,ASSETSNC,337341,71622,71623,71622,NaN,1.941400e+14,...,1.733584e+16,Variable.TYPE_NUM,3.706201e+24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,TAXEXP,419038,68849,68850,68849,NaN,1.043900e+13,...,5.243518e+14,Variable.TYPE_NUM,5.621709e+21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,TAXLIABILITIES,425239,32161,32162,32161,NaN,1.092380e+13,...,9.532775e+14,Variable.TYPE_NUM,6.693783e+21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,TBVPS,423896,68853,68854,68853,NaN,2.188325e+09,...,6.366894e+09,Variable.TYPE_NUM,2.305091e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,FUNDAMENTALS,FIDAP_DATA_SOURCES,FIDAP_SCHEMA,TICKER,430420,7212,7212,7212,141.0,NaN,...,NaN,Variable.TYPE_CAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
